Import Libraries

In [21]:
import pandas as pd 
import os
import shutil
import torch
import numpy as np
from torch.utils.data import Dataset
from torchvision import datasets, models, transforms
from torchvision.transforms import ToTensor
import PIL
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.optim as optim
import sklearn
from sklearn import metrics

In [2]:
folder_path = 'Data'
train_path = folder_path+'/train'
val_path = folder_path+'/valid'
test_path = folder_path+'/test'
train_csv = pd.read_csv(train_path+'/_classes.csv')
val_csv = pd.read_csv(val_path+'/_classes.csv')
test_csv = pd.read_csv(test_path+'/_classes.csv')
frame = pd.concat((train_csv,val_csv,test_csv), axis=0, ignore_index=True)
# frame.to_csv(folder_path+'/classes.csv',index=False)
# files = os.listdir(train_path)
# shutil.copytree(train_path, folder_path+'/allData')
# shutil.copytree(test_path, folder_path+'/allData')
# shutil.copytreev(val_path, folder_path+'/allData')
print(train_csv.iloc[0][1:5].values)

[0 0 1 0]


In [3]:
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, csv_path, images_folder, transform = None):
        self.df = pd.read_csv(csv_path)
        self.images_folder = images_folder
        self.transform = transform

    def __len__(self):
        return len(self.df)
    def __getitem__(self, index):
        filename = self.df.loc[index]['filename']
        labels = self.df.iloc[index][1:].values.astype(float)
        image = PIL.Image.open(os.path.join(self.images_folder, filename))
        if self.transform is not None:
            image = self.transform(image)
        return image, torch.tensor(labels)

In [4]:
data_path = folder_path + '/allData'
df = pd.read_csv(folder_path+'/classes.csv')
print(df.head)
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((224, 224)),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
])
dataset = CustomDataset(folder_path+'/classes.csv', data_path, transform=transform)
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [0.9, 0.1])

<bound method NDFrame.head of                                                filename   OK   spaghetti  \
0     stringing_158_jpg.rf.18396c1d56a508411952d165f...    0           0   
1     stringing_111_jpg.rf.18df0b478a32919a335ff2c95...    0           0   
2     spaghetti_32_jpg.rf.19ef7bfb0484134b619e003086...    0           1   
3     OK_37_jpg.rf.17dd174f9b97c1f4a4a13d3ba567971d.jpg    1           0   
4     stringing_122_jpg.rf.1b1bef7f4adc5cfbcee2cd541...    0           0   
...                                                 ...  ...         ...   
2015  spaghetti_112_jpg.rf.f1c6d92a208d3e15197e84e66...    0           1   
2016  OK_68_jpg.rf.fb96a134c9cc26c956167fb36c2cce8c.jpg    1           0   
2017  blobs_90_jpg.rf.fa0652a2dbf773df332b408fc9fc24...    0           0   
2018  OK2_3_jpg.rf.f5e5da9eadb6daa98dfa00fe6e01e695.jpg    1           0   
2019  OK2_15_jpg.rf.f4cd78da0ebdaf958f7eb7fd35b43d8a...    1           0   

       stringing   unsmooth_surface  
0              1   

In [5]:
num_of_classes = 4
model = models.resnet18(pretrained=True)
for param in model.parameters():
    param.requires_grad = False
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, num_of_classes)

for param in model.parameters():
    print(param.requires_grad)


c:\Users\aalqa\anaconda3\envs\cs323\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\aalqa\anaconda3\envs\cs323\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
True
True


In [6]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

trainLoader =  DataLoader(train_dataset, batch_size=32, shuffle=True)
testLoader =  DataLoader(test_dataset, batch_size=32, shuffle=False)


In [17]:
for epoch in range(10):

    running_loss = 0.0
    train_loss_history = []
    for i, data in enumerate(trainLoader):
        inputs, labels = data
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        if i % 10 == 5:
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0

print('Finished Training')

[1,     6] loss: 0.002
[1,    16] loss: 0.003
[1,    26] loss: 0.003
[1,    36] loss: 0.003
[1,    46] loss: 0.003
[1,    56] loss: 0.003
[2,     6] loss: 0.002
[2,    16] loss: 0.003
[2,    26] loss: 0.003
[2,    36] loss: 0.003
[2,    46] loss: 0.003
[2,    56] loss: 0.003
[3,     6] loss: 0.002
[3,    16] loss: 0.003
[3,    26] loss: 0.003
[3,    36] loss: 0.002
[3,    46] loss: 0.003
[3,    56] loss: 0.002
[4,     6] loss: 0.001
[4,    16] loss: 0.002
[4,    26] loss: 0.003
[4,    36] loss: 0.003
[4,    46] loss: 0.002
[4,    56] loss: 0.002
[5,     6] loss: 0.001
[5,    16] loss: 0.002
[5,    26] loss: 0.002
[5,    36] loss: 0.002
[5,    46] loss: 0.002
[5,    56] loss: 0.002
[6,     6] loss: 0.001
[6,    16] loss: 0.002
[6,    26] loss: 0.002
[6,    36] loss: 0.002
[6,    46] loss: 0.002
[6,    56] loss: 0.002
[7,     6] loss: 0.001
[7,    16] loss: 0.002
[7,    26] loss: 0.002
[7,    36] loss: 0.002
[7,    46] loss: 0.002
[7,    56] loss: 0.002
[8,     6] loss: 0.001
[8,    16] 

In [18]:
model.eval()

correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in testLoader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        _, real = torch.max(labels, 1)
        total += labels.size(0)
        print(real)
        correct += (predicted == real).sum().item()

accuracy = correct / total * 100
print(f'Test Accuracy: {accuracy:.2f}%')
        

tensor([3, 2, 3, 0, 0, 3, 3, 0, 3, 2, 3, 1, 2, 3, 1, 0, 3, 1, 0, 0, 0, 3, 1, 3,
        2, 0, 0, 3, 3, 3, 3, 2])
tensor([2, 3, 2, 1, 2, 2, 2, 3, 0, 3, 0, 1, 2, 3, 2, 0, 0, 2, 3, 0, 2, 0, 2, 1,
        2, 2, 2, 3, 2, 3, 3, 3])
tensor([3, 0, 3, 0, 0, 1, 3, 0, 0, 0, 2, 3, 0, 2, 2, 3, 2, 3, 1, 2, 0, 0, 0, 3,
        0, 1, 1, 1, 0, 3, 2, 0])
tensor([3, 3, 0, 0, 2, 1, 3, 2, 0, 0, 2, 0, 2, 0, 0, 1, 1, 0, 2, 0, 3, 0, 2, 0,
        0, 3, 3, 0, 3, 0, 2, 0])
tensor([0, 0, 2, 0, 0, 3, 0, 2, 3, 3, 2, 1, 3, 1, 3, 2, 2, 2, 0, 3, 3, 2, 0, 3,
        3, 3, 3, 3, 3, 0, 1, 1])
tensor([3, 0, 1, 2, 1, 3, 2, 2, 0, 2, 2, 1, 2, 3, 3, 3, 3, 2, 0, 2, 2, 2, 0, 3,
        0, 1, 2, 0, 3, 2, 0, 2])
tensor([2, 0, 2, 0, 0, 0, 2, 0, 2, 1])
Test Accuracy: 83.17%
